# Oficina de dados abertos do Transferegov

O foco da oficina será na parte das emendas parlamentares chamadas "emendas Pix" que é operacionaliza pelo Transferegov, no módulo de Transferências Especiais. Os dados estão disponibilizados por meio de API de dados abertos.

Endereço da API: https://docs.api.transferegov.gestao.gov.br/transferenciasespeciais/#/plano_acao_especial/get_plano_acao_especial

## Plano de ação especial

Vamos carregar em uma única tabela todos os dados de Plano de Ação Especial.

### Baixar os dados

In [56]:
import pandas as pd
import requests
from tqdm.notebook import tqdm

In [33]:
LIMIT = 1000
ANO = 2024
URL = "https://api.transferegov.gestao.gov.br/transferenciasespeciais/plano_acao_especial"

In [26]:
content = requests.get(
    URL,
    params={
        "ano_emenda_parlamentar_plano_acao": f"eq.{ANO}",
        "limit": 10,
    },
    headers={
        "Prefer": "count=exact",
    }
)

In [27]:
content.headers

{'Date': 'Mon, 25 Nov 2024 18:21:05 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Range': '0-9/10934', 'Content-Location': '/plano_acao_especial?ano_emenda_parlamentar_plano_acao=eq.2024&limit=10', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'Content-Security-Policy': "default-src 'self'"}

Vamos descobrir a quantidade de registros para iterar sobre ela.

In [32]:
record_count = int(content.headers["Content-Range"].split("/")[-1])
record_count

10934

In [17]:
data = content.json()

In [29]:
df = pd.DataFrame(columns=data[0].keys())

In [35]:
df.columns

Index(['id_plano_acao', 'codigo_plano_acao', 'ano_plano_acao',
       'modalidade_plano_acao', 'situacao_plano_acao',
       'cnpj_beneficiario_plano_acao', 'nome_beneficiario_plano_acao',
       'uf_beneficiario_plano_acao', 'codigo_banco_plano_acao',
       'codigo_situacao_dado_bancario_plano_acao', 'nome_banco_plano_acao',
       'numero_agencia_plano_acao', 'dv_agencia_plano_acao',
       'numero_conta_plano_acao', 'dv_conta_plano_acao',
       'nome_parlamentar_emenda_plano_acao',
       'ano_emenda_parlamentar_plano_acao',
       'codigo_parlamentar_emenda_plano_acao',
       'sequencial_emenda_parlamentar_plano_acao',
       'numero_emenda_parlamentar_plano_acao',
       'codigo_emenda_parlamentar_formatado_plano_acao',
       'codigo_descricao_areas_politicas_publicas_plano_acao',
       'descricao_programacao_orcamentaria_plano_acao',
       'motivo_impedimento_plano_acao', 'valor_custeio_plano_acao',
       'valor_investimento_plano_acao', 'id_programa'],
      dtype='object

In [57]:
pages = []
for offset in tqdm(range(0, record_count, LIMIT)):
    content = requests.get(
        URL,
        params={
            "ano_emenda_parlamentar_plano_acao": f"eq.{ANO}",
            "limit": LIMIT,
            "offset": offset,
        },
    )
    pages.extend(content.json())

df = pd.DataFrame(pages)
df

  0%|          | 0/11 [00:00<?, ?it/s]

,id_plano_acao,codigo_plano_acao,ano_plano_acao,modalidade_plano_acao,situacao_plano_acao,cnpj_beneficiario_plano_acao,nome_beneficiario_plano_acao,uf_beneficiario_plano_acao,codigo_banco_plano_acao,codigo_situacao_dado_bancario_plano_acao,...,codigo_parlamentar_emenda_plano_acao,sequencial_emenda_parlamentar_plano_acao,numero_emenda_parlamentar_plano_acao,codigo_emenda_parlamentar_formatado_plano_acao,codigo_descricao_areas_politicas_publicas_plano_acao,descricao_programacao_orcamentaria_plano_acao,motivo_impedimento_plano_acao,valor_custeio_plano_acao,valor_investimento_plano_acao,id_programa
0,69182,09032024-069182,2024,Especial,CIENTE,95990131000170,MUNICIPIO DE NOVA ITABERABA,SC,104,4.0,...,2855,22,202428550022,202428550022-Pedro Uczai,15-Urbanismo / 451-Infraestrutura Urbana,None,None,0.0,400000.0,21
1,64478,09032024-064478,2024,Especial,CIENTE,41522186000126,MUNICIPIO DE PASSAGEM FRANCA DO PIAUI,PI,001,4.0,...,4496,1,202444960001,202444960001-Jussara Lima,"13-Cultura / 391-Patrimônio Histórico, Artísti...",None,None,0.0,250000.0,21
2,64487,09032024-064487,2024,Especial,CIENTE,01612590000176,MUNICIPIO DE MILTON BRANDAO,PI,104,4.0,...,4496,1,202444960001,202444960001-Jussara Lima,"6-Segurança Pública / 181-Policiamento , 6-Seg...",None,None,0.0,250000.0,21
3,72493,09032024-072493,2024,Especial,CIENTE,18114264000131,MUNICIPIO DE ESPERA FELIZ,MG,104,4.0,...,3731,9,202437310009,202437310009-Misael Varella,"10-Saúde / 301-Atenção Básica , 10-Saúde / 302...",None,None,0.0,500000.0,21
4,73453,09032024-073453,2024,Especial,CIENTE,18114264000131,MUNICIPIO DE ESPERA FELIZ,MG,104,4.0,...,4057,3,202440570003,202440570003-Pinheirinho,"10-Saúde / 301-Atenção Básica , 10-Saúde / 302...",None,None,0.0,2000000.0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10929,74317,09032024-074317,2024,Especial,CIENTE,75368928000122,MUNICIPIO DE MAMBORE,PR,104,4.0,...,4313,1,202443130001,202443130001-Beto Preto,"15-Urbanismo / 451-Infraestrutura Urbana , 20-...",None,None,0.0,670000.0,21
10930,69119,09032024-069119,2024,Especial,CIENTE,14196042000154,TEOLÂNDIA,BA,104,4.0,...,3906,5,202439060005,202439060005-Alex Santana,8-Assistência Social / 244-Assistência Comunit...,None,None,1530737.0,2469263.0,21
10931,73476,09032024-073476,2024,Especial,CIENTE,18094870000132,MUNICIPIO DE SENHORA DOS REMEDIOS,MG,001,4.0,...,4057,3,202440570003,202440570003-Pinheirinho,"15-Urbanismo / 451-Infraestrutura Urbana , 26-...",None,None,0.0,500000.0,21
10932,72488,09032024-072488,2024,Especial,CIENTE,46362927000172,MUNICIPIO DE BROTAS,SP,104,4.0,...,3730,11,202437300011,202437300011-Miguel Lombardi,15-Urbanismo / 451-Infraestrutura Urbana,None,None,0.0,300000.0,21


### Análise

In [43]:
df["nome_parlamentar_emenda_plano_acao"].value_counts()

nome_parlamentar_emenda_plano_acao
Ivete da Silveira           178
Jorge Seif                  167
Ciro Nogueira               103
Luis Carlos Heinze           92
Fred Costa                   91
                           ... 
Camila Jara                   1
Pastor Sargento Isidório      1
Bebeto                        1
Célia Xakriabá                1
Daniella Ribeiro              1
Name: count, Length: 530, dtype: int64

In [46]:
df["valor_emenda"] = df["valor_custeio_plano_acao"] + df["valor_investimento_plano_acao"]

In [52]:
total_emendas_por_parlamentar = df.groupby("nome_parlamentar_emenda_plano_acao").sum()["valor_emenda"]
total_emendas_por_parlamentar

nome_parlamentar_emenda_plano_acao
Abilio Brunini      8485793.0
Acácio Favacho     16732792.0
Adail Filho        18434456.0
Adilson Barroso    15515785.0
Adolfo Viana       15650000.0
                      ...    
Zé Silva           13835792.0
Zé Trovão          14935792.0
Zé Vitor           17671585.0
Átila Lins         15121585.0
Átila Lira         17585585.0
Name: valor_emenda, Length: 530, dtype: float64

In [55]:
total_emendas_por_parlamentar.sort_values(ascending=False)[:20]

nome_parlamentar_emenda_plano_acao
Jussara Lima                 34817425.0
Marcos Rogério               34817425.0
Jayme Campos                 34817425.0
Davi Alcolumbre              34815000.0
Otto Alencar                 34634850.0
Ciro Nogueira                34634850.0
Giordano                     34500000.0
Eliziane Gama                34317425.0
Fernando Farias              34317425.0
Angelo Coronel               34134850.0
Marcelo Castro               33517425.0
Randolfe Rodrigues           33517425.0
Mecias de Jesus              33500000.0
Carlos Fávaro                33332850.0
Dr. Hiran                    33300000.0
Styvenson Valentim           32774850.0
Zenaide Maia                 32360000.0
Luis Carlos Heinze           32017350.0
Professora Dorinha Seabra    31417420.0
Nelsinho Trad                30816850.0
Name: valor_emenda, dtype: float64